<a href="https://colab.research.google.com/github/Sarvagya4/LangChain/blob/main/Langchain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q chromadb==0.4.22
!pip install -q langchain==0.1.4
!pip install -q sentence_transformers==2.3.0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR']= '/content/drive/MyDrive/kaggle'

In [4]:
!kaggle datasets download -d kotartemiy/topic-labeled-news-dataset

Dataset URL: https://www.kaggle.com/datasets/kotartemiy/topic-labeled-news-dataset
License(s): CC0-1.0
topic-labeled-news-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import numpy as np
import pandas as pd

In [6]:
import zipfile

file_path = '/content/topic-labeled-news-dataset.zip'

In [7]:
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kaggle')

In [8]:
news = pd.read_csv('/content/drive/MyDrive/kaggle/labelled_newscatcher_dataset.csv', sep=';')
MAX_NEWS = 1000
DOCUMENT="title"
TOPIC="topic"
subset_news = news.head(MAX_NEWS)

In [9]:
news.head(2)

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en


In [10]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [11]:
df_loader = DataFrameLoader(subset_news, page_content_column=DOCUMENT)

In [12]:
df_document = df_loader.load()

In [13]:
display(df_document[:2])

[Document(page_content="A closer look at water-splitting's solar fuel potential", metadata={'topic': 'SCIENCE', 'link': 'https://www.eurekalert.org/pub_releases/2020-08/dbnl-acl080620.php', 'domain': 'eurekalert.org', 'published_date': '2020-08-06 13:59:45', 'lang': 'en'}),
 Document(page_content='An irresistible scent makes locusts swarm, study finds', metadata={'topic': 'SCIENCE', 'link': 'https://www.pulse.ng/news/world/an-irresistible-scent-makes-locusts-swarm-study-finds/jy784jw', 'domain': 'pulse.ng', 'published_date': '2020-08-12 15:14:19', 'lang': 'en'})]

In [14]:
from langchain.text_splitter import CharacterTextSplitter

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=10)
texts = text_splitter.split_documents(df_document)

In [16]:
display(texts[:2])


[Document(page_content="A closer look at water-splitting's solar fuel potential", metadata={'topic': 'SCIENCE', 'link': 'https://www.eurekalert.org/pub_releases/2020-08/dbnl-acl080620.php', 'domain': 'eurekalert.org', 'published_date': '2020-08-06 13:59:45', 'lang': 'en'}),
 Document(page_content='An irresistible scent makes locusts swarm, study finds', metadata={'topic': 'SCIENCE', 'link': 'https://www.pulse.ng/news/world/an-irresistible-scent-makes-locusts-swarm-study-finds/jy784jw', 'domain': 'pulse.ng', 'published_date': '2020-08-12 15:14:19', 'lang': 'en'})]

In [17]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [18]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    texts, embedding_function, persist_directory=directory_cdb
)

In [19]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

In [20]:
retriever = chroma_db.as_retriever()

In [21]:
model_id = "databricks/dolly-v2-3b"
task="text-generation"

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
hf_llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task=task,
    pipeline_kwargs={
        "max_new_tokens": 256,
        "repetition_penalty":1.1,
        "return_full_text":True
    },
)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


In [ ]:
document_qa = RetrievalQA.from_chain_type(
    llm=hf_llm, retriever=retriever, chain_type='stuff'
)

In [ ]:
response = document_qa.invoke("Can I buy a Toshiba laptop?")

display(response)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | hf_llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("Can I buy a Toshiba laptop?")

In [ ]:
embedding_s1 = embedding_function.embed_query(
    "I would like to eat more vegetables and exercise every day")

embedding_s2 = embedding_function.embed_query(
    "I will try to maintain a healthier lifestyle.")

embedding_s3 = embedding_function.embed_query(
    "I prefer to play football")

In [ ]:
print(f""" embedding_s1 = {len(embedding_s1)}
 embedding_s2 =  {len(embedding_s2)}
 embedding_s3 =  {len(embedding_s3)}""")

In [ ]:
print(embedding_s1[:5])
print(embedding_s2[:5])
print(embedding_s3[:5])

In [ ]:
!pip install -q scikit-learn==1.2.2

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
embedding_s1_2d = np.array(embedding_s1).reshape(1, -1)
embedding_s2_2d = np.array(embedding_s2).reshape(1, -1)
embedding_s3_2d = np.array(embedding_s3).reshape(1, -1)

In [ ]:
print(embedding_s1_2d[0][:5])

In [ ]:
print(cosine_similarity(embedding_s1_2d, embedding_s2_2d))
print(cosine_similarity(embedding_s1_2d, embedding_s3_2d))
print(cosine_similarity(embedding_s2_2d, embedding_s3_2d))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

PCA_model = PCA(n_components = 2)

In [ ]:
embeddings_sentences=[]
embeddings_sentences.append(embedding_s1)
embeddings_sentences.append(embedding_s2)
embeddings_sentences.append(embedding_s3)
PCA_model.fit(embeddings_sentences)
embeddings_coord = PCA_model.transform(embeddings_sentences)

In [ ]:
print(embeddings_coord)

In [ ]:
import matplotlib.pyplot as plt

x, y = zip(*embeddings_coord)

# Name the points
names = ['s1', 's2', 's3']
# Colors
colors = ['red', 'green', 'blue']

for i, name in enumerate(names):
    plt.scatter(x[i], y[i], marker='o', color=colors[i], label=name)

plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()